In [5]:
#import libraries
import pandas as pd
import numpy as np
import datetime as dt
import pycountry
import spacy
import re
from geopy.geocoders import Nominatim

In [6]:
#importing data
data=pd.read_csv("Airplane_Crashes_and_Fatalities_Since_1908.csv")

In [7]:
data

,Date,Time,Location,Operator,Flight #,Route,Type,Registration,cn/In,Aboard,Fatalities,Ground,Summary
0,09/17/1908,17:18,"Fort Myer, Virginia",Military - U.S. Army,NaN,Demonstration,Wright Flyer III,NaN,1,2.0,1.0,0.0,"During a demonstration flight, a U.S. Army fly..."
1,07/12/1912,06:30,"AtlantiCity, New Jersey",Military - U.S. Navy,NaN,Test flight,Dirigible,NaN,NaN,5.0,5.0,0.0,First U.S. dirigible Akron exploded just offsh...
2,08/06/1913,NaN,"Victoria, British Columbia, Canada",Private,-,NaN,Curtiss seaplane,NaN,NaN,1.0,1.0,0.0,The first fatal airplane accident in Canada oc...
3,09/09/1913,18:30,Over the North Sea,Military - German Navy,NaN,NaN,Zeppelin L-1 (airship),NaN,NaN,20.0,14.0,0.0,The airship flew into a thunderstorm and encou...
4,10/17/1913,10:30,"Near Johannisthal, Germany",Military - German Navy,NaN,NaN,Zeppelin L-2 (airship),NaN,NaN,30.0,30.0,0.0,Hydrogen gas which was being vented was sucked...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5263,05/20/2009,06:30,"Near Madiun, Indonesia",Military - Indonesian Air Force,NaN,Jakarta - Maduin,Lockheed C-130 Hercules,A-1325,1982,112.0,98.0,2.0,"While on approach, the military transport cras..."
5264,05/26/2009,NaN,"Near Isiro, DemocratiRepubliCongo",Service Air,NaN,Goma - Isiro,Antonov An-26,9Q-CSA,5005,4.0,4.0,NaN,The cargo plane crashed while on approach to I...
5265,06/01/2009,00:15,"AtlantiOcean, 570 miles northeast of Natal, Br...",Air France,447,Rio de Janeiro - Paris,Airbus A330-203,F-GZCP,660,228.0,228.0,0.0,The Airbus went missing over the AtlantiOcean ...
5266,06/07/2009,08:30,"Near Port Hope Simpson, Newfoundland, Canada",Strait Air,NaN,Lourdes de BlanSablon - Port Hope Simpson,Britten-Norman BN-2A-27 Islander,C-FJJR,424,1.0,1.0,0.0,The air ambulance crashed into hills while att...


In [8]:
data.dtypes

Date             object
Time             object
Location         object
Operator         object
Flight #         object
Route            object
Type             object
Registration     object
cn/In            object
Aboard          float64
Fatalities      float64
Ground          float64
Summary          object
dtype: object

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5268 entries, 0 to 5267
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          5268 non-null   object 
 1   Time          3049 non-null   object 
 2   Location      5248 non-null   object 
 3   Operator      5250 non-null   object 
 4   Flight #      1069 non-null   object 
 5   Route         3561 non-null   object 
 6   Type          5241 non-null   object 
 7   Registration  4933 non-null   object 
 8   cn/In         4040 non-null   object 
 9   Aboard        5246 non-null   float64
 10  Fatalities    5256 non-null   float64
 11  Ground        5246 non-null   float64
 12  Summary       4878 non-null   object 
dtypes: float64(3), object(10)
memory usage: 535.2+ KB


In [10]:
#change the Date type column to datetime
data.Date = pd.to_datetime(data.Date)

In [11]:
data.dtypes

Date            datetime64[ns]
Time                    object
Location                object
Operator                object
Flight #                object
Route                   object
Type                    object
Registration            object
cn/In                   object
Aboard                 float64
Fatalities             float64
Ground                 float64
Summary                 object
dtype: object

In [12]:
data.isnull().sum()

Date               0
Time            2219
Location          20
Operator          18
Flight #        4199
Route           1707
Type              27
Registration     335
cn/In           1228
Aboard            22
Fatalities        12
Ground            22
Summary          390
dtype: int64

In [13]:
data['Location'].fillna('Unknown', inplace=True)
data['Operator'].fillna('unknown', inplace=True)
data['Route'].fillna('Unknown', inplace=True)
data['Type'].fillna('unknown', inplace=True)
data['Registration'].fillna(data['Registration'].mode()[0], inplace=True)
data['cn/In'].fillna(data['cn/In'][0], inplace=True)
data['Aboard'].fillna(data['Aboard'][0], inplace=True)
data['Fatalities'].fillna(data['Fatalities'][0], inplace=True)
data['Ground'].fillna(data['Ground'][0], inplace=True)
data['Summary'].fillna('unknown', inplace=True)
# Replace missing time values with midnight (00:00:00)
default_time = dt.time(0, 0)
data['Time'].fillna(default_time, inplace=True)
#drop flight column
data.drop('Flight #',axis=1,inplace=True)

In [14]:
data.isnull().sum()

Date            0
Time            0
Location        0
Operator        0
Route           0
Type            0
Registration    0
cn/In           0
Aboard          0
Fatalities      0
Ground          0
Summary         0
dtype: int64

In [15]:
data

,Date,Time,Location,Operator,Route,Type,Registration,cn/In,Aboard,Fatalities,Ground,Summary
0,1908-09-17,17:18,"Fort Myer, Virginia",Military - U.S. Army,Demonstration,Wright Flyer III,49,1,2.0,1.0,0.0,"During a demonstration flight, a U.S. Army fly..."
1,1912-07-12,06:30,"AtlantiCity, New Jersey",Military - U.S. Navy,Test flight,Dirigible,49,1,5.0,5.0,0.0,First U.S. dirigible Akron exploded just offsh...
2,1913-08-06,00:00:00,"Victoria, British Columbia, Canada",Private,Unknown,Curtiss seaplane,49,1,1.0,1.0,0.0,The first fatal airplane accident in Canada oc...
3,1913-09-09,18:30,Over the North Sea,Military - German Navy,Unknown,Zeppelin L-1 (airship),49,1,20.0,14.0,0.0,The airship flew into a thunderstorm and encou...
4,1913-10-17,10:30,"Near Johannisthal, Germany",Military - German Navy,Unknown,Zeppelin L-2 (airship),49,1,30.0,30.0,0.0,Hydrogen gas which was being vented was sucked...
...,...,...,...,...,...,...,...,...,...,...,...,...
5263,2009-05-20,06:30,"Near Madiun, Indonesia",Military - Indonesian Air Force,Jakarta - Maduin,Lockheed C-130 Hercules,A-1325,1982,112.0,98.0,2.0,"While on approach, the military transport cras..."
5264,2009-05-26,00:00:00,"Near Isiro, DemocratiRepubliCongo",Service Air,Goma - Isiro,Antonov An-26,9Q-CSA,5005,4.0,4.0,0.0,The cargo plane crashed while on approach to I...
5265,2009-06-01,00:15,"AtlantiOcean, 570 miles northeast of Natal, Br...",Air France,Rio de Janeiro - Paris,Airbus A330-203,F-GZCP,660,228.0,228.0,0.0,The Airbus went missing over the AtlantiOcean ...
5266,2009-06-07,08:30,"Near Port Hope Simpson, Newfoundland, Canada",Strait Air,Lourdes de BlanSablon - Port Hope Simpson,Britten-Norman BN-2A-27 Islander,C-FJJR,424,1.0,1.0,0.0,The air ambulance crashed into hills while att...


In [16]:
# Type Operator ( Military / Non Military)

# Filtrer les lignes où la colonne "Operator" contient la chaîne "Military"
military_df = data[data['Operator'].str.contains('Military')]

# Filtrer les lignes où la colonne "Operator" ne contient pas la chaîne "Military"
non_military_df = data[~data['Operator'].str.contains('Military')]

# Ajouter une nouvelle colonne "Type Operator" en fonction de la condition
data['Type Operator'] = data['Operator'].apply(lambda x: 'Military' if 'Military' in x else 'Non Military')



In [17]:
#data mining for the countries
nlp = spacy.load("en_core_web_sm")
country_names = {country.name for country in pycountry.countries} | {'England', 'Mali','Russia','The Netherland','Bulgeria','Nevada','Scotland','Vietnam','Bolivia','Venezuela','Bulgeria','Finland','Yugoslavia','Romainia','Wales','Cyprus','Benin','Tanzania','Nepal','Baangladesh','Cameroon','Iran','Botswana','Ohio','Mozambique','Philipines','South Africa','Taiwan','Oman','South Korea','Soviet Union','Afghanstan','Congo','DemocratiRepubliCongo','Yugoslavia','Morocco','Niger','Bahrain','Chili','North Korea','Domincan Republic','Gabon','Cuba','Uruguay',''}

In [18]:
# Function to extract the country from the "Country" column
def extract_country(location):
    # Check if the location is in the set of country names
    if location in country_names:
        return location

    # Apply spaCy NER
    doc = nlp(location)

    # Extract GPE (Geopolitical Entity) entities and check if it's a country
    for ent in doc.ents:
        if ent.label_ == "GPE" and ent.text in country_names:
            return ent.text

    # Return None if no country is found
    return None

# Apply the function to create a new column "Country"
data['Country'] = data['Location'].apply(extract_country)

df = data[['Date','Time','Location','Country','Operator','Type Operator','Route','Type','Registration','cn/In','Aboard','Fatalities','Ground','Summary']]


In [19]:
# Save the final output data to a CSV file
df.to_csv('datacleaned.csv', index=False)